In [1]:
import os

os.environ["PROJECT_ID"] = "qwiklabs-gcp-04-133e595cc3fe"

In [7]:
import os
from datetime import datetime

import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google import experimental
from task import train

# Set variables
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
PROJECT = "qwiklabs-gcp-04-133e595cc3fe"
BUCKET = "physionet_2009"
PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root"
REGION = "us-central1"

OUTDIR = f"gs://{BUCKET}/models/trained_xgb_model_{TIMESTAMP}"
MODEL_DISPLAY_NAME = f"xgb_model_{TIMESTAMP}"

#PYTHON_PACKAGE_URIS = f"gs://{BUCKET}/model_pkgs/abp_xgboost_trainer-0.1.tar.gz"
PYTHON_PACKAGE_URIS = f"gs://{BUCKET}/model_pkgs/waveform_fun-0.0.tar.gz"
MACHINE_TYPE = "n1-standard-16"
REPLICA_COUNT = 1
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-5:latest"
)
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest"
)
PYTHON_MODULE = "waveform_fun.models.xgb_trainer.task"

%env PROJECT=$PROJECT
%env BUCKET=$BUCKET
%env REGION=$REGION
%env MODEL_DISPLAY_NAME=$MODEL_DISPLAY_NAME
%env PYTHON_PACKAGE_URIS=$PYTHON_PACKAGE_URIS
%env MACHINE_TYPE=$MACHINE_TYPE
%env REPLICA_COUNT=$REPLICA_COUNT
%env PYTHON_PACKAGE_EXECUTOR_IMAGE_URI=$PYTHON_PACKAGE_EXECUTOR_IMAGE_URI
%env PYTHON_MODULE=$PYTHON_MODULE

env: PROJECT=qwiklabs-gcp-04-133e595cc3fe
env: BUCKET=physionet_2009
env: REGION=us-central1
env: MODEL_DISPLAY_NAME=xgb_model_20211110155452
env: PYTHON_PACKAGE_URIS=gs://physionet_2009/model_pkgs/waveform_fun-0.0.tar.gz
env: MACHINE_TYPE=n1-standard-16
env: REPLICA_COUNT=1
env: PYTHON_PACKAGE_EXECUTOR_IMAGE_URI=us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-5:latest
env: PYTHON_MODULE=waveform_fun.models.xgb_trainer.task


In [8]:
%%bash

gcloud beta ai custom-jobs create \
  --region=$REGION \
  --display-name=$MODEL_DISPLAY_NAME \
  --python-package-uris=$PYTHON_PACKAGE_URIS \
  --worker-pool-spec=machine-type=$MACHINE_TYPE,replica-count=$REPLICA_COUNT,executor-image-uri=$PYTHON_PACKAGE_EXECUTOR_IMAGE_URI,python-module=$PYTHON_MODULE

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/741350817607/locations/us-central1/customJobs/2717656293304172544] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud beta ai custom-jobs describe projects/741350817607/locations/us-central1/customJobs/2717656293304172544

or continue streaming the logs with the command

  $ gcloud beta ai custom-jobs stream-logs projects/741350817607/locations/us-central1/customJobs/2717656293304172544


In [14]:
!gcloud beta ai custom-jobs describe projects/741350817607/locations/us-central1/customJobs/2717656293304172544

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
createTime: '2021-11-10T15:54:56.999926Z'
displayName: xgb_model_20211110155452
endTime: '2021-11-10T16:00:36Z'
jobSpec:
  workerPoolSpecs:
  - diskSpec:
      bootDiskSizeGb: 100
      bootDiskType: pd-ssd
    machineSpec:
      machineType: n1-standard-16
    pythonPackageSpec:
      executorImageUri: us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-5:latest
      packageUris:
      - gs://physionet_2009/model_pkgs/waveform_fun-0.0.tar.gz
      pythonModule: waveform_fun.models.xgb_trainer.task
    replicaCount: '1'
name: projects/741350817607/locations/us-central1/customJobs/2717656293304172544
startTime: '2021-11-10T15:59:35Z'
state: JOB_STATE_SUCCEEDED
updateTime: '2021-11-10T16:00:37.504052Z'


In [13]:
!gsutil cp gs://physionet_2009/model_pkgs/waveform_fun-0.0.tar.gz .

Copying gs://physionet_2009/model_pkgs/waveform_fun-0.0.tar.gz...
/ [1 files][  9.8 KiB/  9.8 KiB]                                                
Operation completed over 1 objects/9.8 KiB.                                      
